## Tubería con Auto Loader para reportes mensuales.

### Acceso a Blob Storage:

In [0]:
import pyspark

In [0]:
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

### Leer archivo y obtener esquema general

Auto Loader

In [0]:
spark.sql("set spark.sql.streaming.schemaInference=true")

Out[3]: DataFrame[key: string, value: string]

In [0]:
stream_in = (spark.readStream
                .format("cloudFiles")
                .option("cloudFiles.useNotifications", False)
                .option("cloudFiles.format", "csv")
                .option("cloudFiles.schemaLocation", f"wasbs://monthlyreportschema@{storage_account_name}.blob.core.windows.net/")
                .load(f"wasbs://monthlyreports@{storage_account_name}.blob.core.windows.net/"))

### Transformación

In [0]:
from pyspark.sql.functions import *

In [0]:
#Renombrar columnas.

df = stream_in.toDF(*["date", "license_class", "trips_per_day", "farebox_per_day", "unique_drivers", "unique_vehicles", "vehicles_per_day", "avg_days_vehicles_on_road", "avg_hours_per_day_per_vehicle", "avg_days_drivers_on_road", "avg_hours_per_day_per_driver", "avg_minutes_per_trip", "percent_of_trips_paid_with_credit_card", "trips_per_day_shared", "rescued_data"])

In [0]:
#Convertir las columnas a minúscula.
df = df.select("*", lower(col("license_class")))

df = df.select("*", lower(col("rescued_data")))

In [0]:
#Reorganizar tabla p1.
df = df.drop("license_class")
df = df.drop("rescued_data")

In [0]:
#Reorganizar tabla p2.
df = df.select(["date", "lower(license_class)", "trips_per_day", "farebox_per_day", "unique_drivers", "unique_vehicles", "vehicles_per_day", "avg_days_vehicles_on_road", "avg_hours_per_day_per_vehicle", "avg_days_drivers_on_road", "avg_hours_per_day_per_driver", "avg_minutes_per_trip", "percent_of_trips_paid_with_credit_card", "trips_per_day_shared", "lower(rescued_data)"])

In [0]:
#Renombrar columnas.
df = df.toDF(*["date", "license_class", "trips_per_day", "farebox_per_day", "unique_drivers", "unique_vehicles", "vehicles_per_day", "avg_days_vehicles_on_road", "avg_hours_per_day_per_vehicle", "avg_days_drivers_on_road", "avg_hours_per_day_per_driver", "avg_minutes_per_trip", "percent_of_trips_paid_with_credit_card", "trips_per_day_shared", "rescued_data"])

In [0]:
#Limpiar espacios de inicio y final con trim.
df = df.withColumn("farebox_per_day", trim(col("farebox_per_day")))

df = df.withColumn("unique_drivers", trim(col("unique_drivers")))

df = df.withColumn("unique_vehicles", trim(col("unique_vehicles")))

df = df.withColumn("vehicles_per_day", trim(col("vehicles_per_day")))

df = df.withColumn("avg_days_vehicles_on_road", trim(col("avg_days_vehicles_on_road")))

df = df.withColumn("avg_hours_per_day_per_vehicle", trim(col("avg_hours_per_day_per_vehicle")))

df = df.withColumn("avg_days_drivers_on_road", trim(col("avg_days_drivers_on_road")))

df = df.withColumn("avg_hours_per_day_per_driver", trim(col("avg_hours_per_day_per_driver")))

df = df.withColumn("avg_minutes_per_trip", trim(col("avg_minutes_per_trip")))

df = df.withColumn("percent_of_trips_paid_with_credit_card", trim(col("percent_of_trips_paid_with_credit_card")))

df = df.withColumn("trips_per_day_shared", trim(col("trips_per_day_shared")))

df = df.withColumn("rescued_data", trim(col("rescued_data")))

In [0]:
#Reemplazar algunos valores.
df = df.withColumn("percent_of_trips_paid_with_credit_card", regexp_replace("percent_of_trips_paid_with_credit_card", "%", ""))

df = df.withColumn("percent_of_trips_paid_with_credit_card", regexp_replace("percent_of_trips_paid_with_credit_card", "-", "0.0"))

df = df.withColumn("trips_per_day", regexp_replace("trips_per_day", ",", ""))

df = df.withColumn("unique_drivers", regexp_replace("unique_drivers", ",", ""))

df = df.withColumn("unique_vehicles", regexp_replace("unique_vehicles", ",", ""))

df = df.withColumn("vehicles_per_day", regexp_replace("vehicles_per_day", ",", ""))

df = df.withColumn("farebox_per_day", regexp_replace("farebox_per_day", "-", "0"))

df = df.withColumn("farebox_per_day", regexp_replace("farebox_per_day", ",", ""))

df = df.withColumn("avg_minutes_per_trip", regexp_replace("avg_minutes_per_trip", "-", "0"))

df = df.withColumn("trips_per_day_shared", regexp_replace("trips_per_day_shared", "-", "0"))

df = df.withColumn("trips_per_day_shared", regexp_replace("trips_per_day_shared", ",", ""))

In [0]:
#Convertir columna de porcentaje a decimal.
df = df.withColumn("percent_of_trips_paid_with_credit_card", df.percent_of_trips_paid_with_credit_card/100)

In [0]:
#Llenar nulos con 0.
df = df.fillna(0, subset = ["trips_per_day", "farebox_per_day", "unique_drivers", "unique_vehicles", "vehicles_per_day", "avg_days_vehicles_on_road", "avg_hours_per_day_per_vehicle", "avg_days_drivers_on_road", "avg_hours_per_day_per_driver", "avg_minutes_per_trip", "percent_of_trips_paid_with_credit_card", "trips_per_day_shared"])

In [0]:
from pyspark.sql.types import *

#Convertir columnas a tipo numérico.
df = df.withColumn("date", df.date.cast(DateType()))

df = df.withColumn("license_class", df.license_class.cast(StringType()))

df = df.withColumn("trips_per_day", df.trips_per_day.cast(IntegerType()))

df = df.withColumn("farebox_per_day", df.farebox_per_day.cast(IntegerType()))

df = df.withColumn("unique_drivers", df.unique_drivers.cast(IntegerType()))

df = df.withColumn("unique_vehicles", df.unique_vehicles.cast(IntegerType()))

df = df.withColumn("vehicles_per_day", df.vehicles_per_day.cast(IntegerType()))

df = df.withColumn("avg_days_vehicles_on_road", df.avg_days_vehicles_on_road.cast(DoubleType()))

df = df.withColumn("avg_hours_per_day_per_vehicle", df.avg_hours_per_day_per_vehicle.cast(DoubleType()))

df = df.withColumn("avg_days_drivers_on_road", df.avg_days_drivers_on_road.cast(DoubleType()))

df = df.withColumn("avg_hours_per_day_per_driver", df.avg_hours_per_day_per_driver.cast(DoubleType()))

df = df.withColumn("avg_minutes_per_trip", df.avg_minutes_per_trip.cast(DoubleType()))

df = df.withColumn("percent_of_trips_paid_with_credit_card", df.percent_of_trips_paid_with_credit_card.cast(DoubleType()))

df = df.withColumn("trips_per_day_shared", df.trips_per_day_shared.cast(IntegerType()))

In [0]:
#Filtrar para que sólo sean los taxis amarillos.
df = df.filter(df.license_class == "yellow")

In [0]:
#Nuevas columnas basadas en tiempo.
df = df.withColumn("month_year1", df.date)

df = df.withColumn("last_day", last_day(df.date))

df = df.withColumn("days", dayofmonth(df.last_day))

df = df.drop("last_day")

df = df.withColumn("trips_per_month", df.trips_per_day * df.days)

In [0]:
#Convertir columna "date" de nuevo a String y modificar.
df = df.withColumn("date", date_format(df.date, "yyyy-MM"))

In [0]:
#Agregar "-28" al string y crear columna "month_date".
df = df.withColumn("month_date", (concat(col("date"), lit("-28"))))


In [0]:
#Columna farebox_per_month.
df = df.withColumn("farebox_per_month", df.farebox_per_day * df.days)

In [0]:
#Convertir "month_date" a datetype() y obtener sólo las semanas.
df = df.withColumn("week", df.month_date.cast(DateType()))

df = df.withColumn("week", weekofyear(col("week")))

In [0]:
#Columna "trips_per_week"
df = df.withColumn("trips_per_week", df.trips_per_day * 7)

In [0]:
#Crear columna de año.
df = df.withColumn("year", year(df.month_date))

In [0]:
#Más columnas.
df = df.withColumn("farebox_per_week", df.farebox_per_day * 7)

df = df.withColumn("total_trips_per_day", df.trips_per_day + df.trips_per_day_shared)

df = df.withColumn("shared_trips_per_day_percent", df.trips_per_day_shared / df.total_trips_per_day)

df = df.withColumn("farebox_per_trip", df.farebox_per_month / df.trips_per_month)

df = df.withColumn("farebox_per_vehicle", df.farebox_per_month / df.unique_vehicles)

df = df.withColumn("trips_per_month_on_road", df.trips_per_day * df.avg_days_vehicles_on_road)

df = df.withColumn("farebox_per_month_on_road", df.farebox_per_day * df.avg_days_vehicles_on_road)

df = df.withColumn("daily_farebox_per_trip", df.farebox_per_day / df.total_trips_per_day)

df = df.withColumn("daily_farebox_per_vehicle", df.farebox_per_day / df.vehicles_per_day)

df = df.withColumn("drivers_vehicles_ratio", df.unique_drivers / df.unique_vehicles)

df = df.withColumn("monthly_farebox_per_vehicle_on_road", df.farebox_per_month_on_road / df.unique_vehicles)

df = df.withColumn("monthly_farebox_per_trip_on_road", df.farebox_per_month_on_road / df.trips_per_month_on_road)

df = df.withColumn("monthly_trips_per_vehicle_on_road", df.trips_per_month_on_road / df.unique_vehicles)

In [0]:
#Eliminar la columna de "days"
df = df.drop("days")

### Carga a SQL Database

In [0]:
def output_sqldb(batch_df, batch_id):

    #Propiedades para la conexión.
    jdbcHostname = "pfservidor.database.windows.net"
    jdbcPort = 1433
    jdbcDatabase = "yellowtaxis"
    jdbcUsername = "ivnng"
    jdbcPassword = "henryPF2023"
    jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

    jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}"

    dbtable = "monthlyreports"

    #Convertir de PySpark DataFrame a Azure DW...
    batch_df.write.format("jdbc").mode("append").option("url", jdbcUrl).option("dbtable", dbtable).save()

Auto Loader

In [0]:
stream_out = (df.writeStream
                .trigger(once = True)
                .foreachBatch(output_sqldb)
                .option("checkpointLocation", "wasbs://reportscheckpoint@pfcrudo.blob.core.windows.net/")
                .start())